# ETL Project
### ***NFL Draft Analysis***

In [1]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

##### Create secrets.py file with username and password to pgadmin

In [2]:
#Import username and password from secrets.py file
from secrets import username, password, database_name

# Extract

##### Extract the data from Wikipedia and  www.pro-football-reference.com. 
* Note: Since we are scraping data from tables we are using pandas otherwise we were going to need to use BeatifulSoup and Splinter

### Extract Combine Info from 2016-2020

In [3]:
#Assign Combine URL's to variable
combine_url_2016 =  'https://www.pro-football-reference.com/draft/2016-combine.htm'
combine_url_2017 = 'https://www.pro-football-reference.com/draft/2017-combine.htm'
combine_url_2018 = 'https://www.pro-football-reference.com/draft/2018-combine.htm'
combine_url_2019 = 'https://www.pro-football-reference.com/draft/2019-combine.htm'
combine_url_2020 = 'https://www.pro-football-reference.com/draft/2020-combine.htm#combine'

In [4]:
#Use Pandas to read in html tables from combine URL's
combine_2016 = pd.read_html(combine_url_2016)
combine_2017 = pd.read_html(combine_url_2017)
combine_2018 = pd.read_html(combine_url_2018)
combine_2019 = pd.read_html(combine_url_2019)
combine_2020 = pd.read_html(combine_url_2020)

In [5]:
#Preview Combine 2016
combine_2016

[             Player Pos          School        College    Ht   Wt  40yd  \
 0    Mehdi Abdesmad  DE     Boston Col.  College Stats   6-6  284  5.10   
 1      Vernon Adams  QB          Oregon  College Stats  5-11  200  4.83   
 2      Jerell Adams  TE  South Carolina  College Stats   6-5  247  4.64   
 3    Bralon Addison  WR          Oregon  College Stats   5-9  197  4.66   
 4    Roberto Aguayo   K   Florida State  College Stats   6-0  207  4.96   
 ..              ...  ..             ...            ...   ...  ...   ...   
 333    Daryl Worley  CB   West Virginia  College Stats   6-1  204  4.64   
 334  Connor Wujciak  DT     Boston Col.  College Stats   6-2  291  4.91   
 335     Tavon Young  CB          Temple  College Stats   5-9  183  4.46   
 336     Avery Young  OT          Auburn  College Stats   6-5  328  5.39   
 337  Anthony Zettel  DT      Penn State  College Stats   6-4  277  4.81   
 
     Vertical Bench Broad Jump 3Cone Shuttle  \
 0       29.5    25        108  7.55  

In [6]:
#Extract the 0th HTML Table to get the necessary combine info
combine_df_2016 = combine_2016[0]
combine_df_2017 = combine_2017[0]
combine_df_2018 = combine_2018[0]
combine_df_2019 = combine_2019[0]
combine_df_2020 = combine_2020[0]

In [7]:
#Preiew the 2016 Combine DF - Combine Extraction Complete!
combine_df_2016.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr)
0,Mehdi Abdesmad,DE,Boston Col.,College Stats,6-6,284,5.10,29.5,25,108,7.55,4.62,NaN
1,Vernon Adams,QB,Oregon,College Stats,5-11,200,4.83,29.5,NaN,114,6.82,4.20,NaN
2,Jerell Adams,TE,South Carolina,College Stats,6-5,247,4.64,32.5,NaN,117,7.05,4.31,New York Giants / 6th / 184th pick / 2016
3,Bralon Addison,WR,Oregon,College Stats,5-9,197,4.66,34.5,13,116,6.95,4.14,NaN
4,Roberto Aguayo,K,Florida State,College Stats,6-0,207,4.96,NaN,NaN,NaN,NaN,NaN,Tampa Bay Buccaneers / 2nd / 59th pick / 2016


### Extract Draft Info from 2016-2020

In [8]:
#Assign Draft URL's to variable
draft_url_2016 = 'https://en.wikipedia.org/wiki/2016_NFL_Draft'
draft_url_2017 = 'https://en.wikipedia.org/wiki/2017_NFL_Draft'
draft_url_2018 = 'https://en.wikipedia.org/wiki/2018_NFL_Draft'
draft_url_2019 = 'https://en.wikipedia.org/wiki/2019_NFL_Draft'
draft_url_2020 = 'https://en.wikipedia.org/wiki/2020_NFL_Draft'

In [9]:
#Use Pandas to read in html tables from draft URL's
draft_2016 = pd.read_html(draft_url_2016)
draft_2017 = pd.read_html(draft_url_2017)
draft_2018 = pd.read_html(draft_url_2018)
draft_2019 = pd.read_html(draft_url_2019)
draft_2020 = pd.read_html(draft_url_2020)

In [10]:
#Preview Draft 2016
draft_2016

[                      2016 NFL Draft                        2016 NFL Draft.1
 0                                NaN                                     NaN
 1                General information                     General information
 2                            Date(s)                       April 28–30, 2016
 3                           Location  Auditorium Theatrein Chicago, Illinois
 4                         Network(s)                ESPN, ESPN2, NFL Network
 5                           Overview                                Overview
 6   253 total selections in 7 rounds        253 total selections in 7 rounds
 7                             League                                     NFL
 8                    First selection          Jared Goff, QBLos Angeles Rams
 9                     Mr. Irrelevant          Kalan Reed, CBTennessee Titans
 10              Most selections (14)                        Cleveland Browns
 11             Fewest selections (5)     New Orleans SaintsCaro

In [11]:
#Extract the 4th HTML Table to get the necessary combine info
draft_df_2016 = draft_2016[4]
draft_df_2017 = draft_2017[4]
draft_df_2018 = draft_2018[4]
draft_df_2019 = draft_2019[4]
draft_df_2020 = draft_2020[4]

In [12]:
#Preview the 2016 Draft DF - Draft Extraction Complete!
draft_df_2016.head()

,Unnamed: 0,Rnd.,Pick No.,NFL team,Player,Pos.,College,Conf.,Notes
0,NaN,1,1.0,Los Angeles Rams,Jared Goff †,QB,California,Pac-12,from Tennessee [R1 - 1]
1,NaN,1,2.0,Philadelphia Eagles,Carson Wentz †,QB,North Dakota State,MVFC,from Cleveland [R1 - 2]
2,NaN,1,3.0,San Diego Chargers,Joey Bosa †,DE,Ohio State,Big Ten,NaN
3,NaN,1,4.0,Dallas Cowboys,Ezekiel Elliott †,RB,Ohio State,Big Ten,NaN
4,NaN,1,5.0,Jacksonville Jaguars,Jalen Ramsey †,CB,Florida State,ACC,NaN


# Transform

### Concatenate and Clean Up Combine Data

In [13]:
#Add year to each combine DF
combine_df_2016['year']='2016'
combine_df_2017['year']='2017'
combine_df_2018['year']='2018'
combine_df_2019['year']='2019'
combine_df_2020['year']='2020'

In [14]:
#Preview 2016 Combine Data with Year Column Added
combine_df_2016.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),year
0,Mehdi Abdesmad,DE,Boston Col.,College Stats,6-6,284,5.10,29.5,25,108,7.55,4.62,NaN,2016
1,Vernon Adams,QB,Oregon,College Stats,5-11,200,4.83,29.5,NaN,114,6.82,4.20,NaN,2016
2,Jerell Adams,TE,South Carolina,College Stats,6-5,247,4.64,32.5,NaN,117,7.05,4.31,New York Giants / 6th / 184th pick / 2016,2016
3,Bralon Addison,WR,Oregon,College Stats,5-9,197,4.66,34.5,13,116,6.95,4.14,NaN,2016
4,Roberto Aguayo,K,Florida State,College Stats,6-0,207,4.96,NaN,NaN,NaN,NaN,NaN,Tampa Bay Buccaneers / 2nd / 59th pick / 2016,2016


In [15]:
#Combine 2016-2020 Combine DF's into 1 DF
combine_frames = [combine_df_2016, combine_df_2017, combine_df_2018, combine_df_2019, combine_df_2020]
combined_combine_df = pd.concat(combine_frames)

In [16]:
#Preview the Combined combine DF
combined_combine_df.head()

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Drafted (tm/rnd/yr),year
0,Mehdi Abdesmad,DE,Boston Col.,College Stats,6-6,284,5.10,29.5,25,108,7.55,4.62,NaN,2016
1,Vernon Adams,QB,Oregon,College Stats,5-11,200,4.83,29.5,NaN,114,6.82,4.20,NaN,2016
2,Jerell Adams,TE,South Carolina,College Stats,6-5,247,4.64,32.5,NaN,117,7.05,4.31,New York Giants / 6th / 184th pick / 2016,2016
3,Bralon Addison,WR,Oregon,College Stats,5-9,197,4.66,34.5,13,116,6.95,4.14,NaN,2016
4,Roberto Aguayo,K,Florida State,College Stats,6-0,207,4.96,NaN,NaN,NaN,NaN,NaN,Tampa Bay Buccaneers / 2nd / 59th pick / 2016,2016


In [17]:
#Drop and Rename Columns
combined_combine_df = combined_combine_df.drop(columns = ['Drafted (tm/rnd/yr)', 'College'])
combined_combine_df = combined_combine_df.rename(columns = {'Player': 'name', 'Ht': 'Height', 'Wt': 'Weight', '40yd': 'Forty_Yard', '3Cone': 'Three_Cone', 'year': 'Year', 'Broad Jump': 'Broad_Jump'})

In [18]:
#Convert Height to Float
def fix(string):
    try:
        feet = int(string.split('-')[0])
    
        inches = int(string.split('-')[1])

    #thats for feet and inches
        return feet * 12 + inches
    except:
        return string
    
combined_combine_df['Height'] = combined_combine_df['Height'].apply(fix)

In [19]:
#Preview the Complete total combine DF - Combine DF Cleanup Complete!
combined_combine_df.head()

,name,Pos,School,Height,Weight,Forty_Yard,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,Year
0,Mehdi Abdesmad,DE,Boston Col.,78,284,5.10,29.5,25,108,7.55,4.62,2016
1,Vernon Adams,QB,Oregon,71,200,4.83,29.5,NaN,114,6.82,4.20,2016
2,Jerell Adams,TE,South Carolina,77,247,4.64,32.5,NaN,117,7.05,4.31,2016
3,Bralon Addison,WR,Oregon,69,197,4.66,34.5,13,116,6.95,4.14,2016
4,Roberto Aguayo,K,Florida State,72,207,4.96,NaN,NaN,NaN,NaN,NaN,2016


### Concatenate and Clean Up Draft Data

In [20]:
#Add year to each draft DF
draft_df_2016['year']='2016'
draft_df_2017['year']='2017'
draft_df_2018['year']='2018'
draft_df_2019['year']='2019'
draft_df_2020['year']='2020'

In [21]:
#Preview the 2016 Draft DF with year column added
draft_df_2016.head()

,Unnamed: 0,Rnd.,Pick No.,NFL team,Player,Pos.,College,Conf.,Notes,year
0,NaN,1,1.0,Los Angeles Rams,Jared Goff †,QB,California,Pac-12,from Tennessee [R1 - 1],2016
1,NaN,1,2.0,Philadelphia Eagles,Carson Wentz †,QB,North Dakota State,MVFC,from Cleveland [R1 - 2],2016
2,NaN,1,3.0,San Diego Chargers,Joey Bosa †,DE,Ohio State,Big Ten,NaN,2016
3,NaN,1,4.0,Dallas Cowboys,Ezekiel Elliott †,RB,Ohio State,Big Ten,NaN,2016
4,NaN,1,5.0,Jacksonville Jaguars,Jalen Ramsey †,CB,Florida State,ACC,NaN,2016


In [22]:
#Combine 2016-2020 Draft DF's into 1 DF
draft_frames = [draft_df_2016, draft_df_2017, draft_df_2018, draft_df_2019, draft_df_2020]
combined_draft_df = pd.concat(draft_frames)

In [23]:
#Preview the combined draft DF
combined_draft_df.head()

,Unnamed: 0,Rnd.,Pick No.,NFL team,Player,Pos.,College,Conf.,Notes,year
0,NaN,1,1.0,Los Angeles Rams,Jared Goff †,QB,California,Pac-12,from Tennessee [R1 - 1],2016
1,NaN,1,2.0,Philadelphia Eagles,Carson Wentz †,QB,North Dakota State,MVFC,from Cleveland [R1 - 2],2016
2,NaN,1,3.0,San Diego Chargers,Joey Bosa †,DE,Ohio State,Big Ten,NaN,2016
3,NaN,1,4.0,Dallas Cowboys,Ezekiel Elliott †,RB,Ohio State,Big Ten,NaN,2016
4,NaN,1,5.0,Jacksonville Jaguars,Jalen Ramsey †,CB,Florida State,ACC,NaN,2016


In [24]:
#Drop and Rename Draft DF Columns
combined_draft_df = combined_draft_df.drop(columns = ['Notes', 'Unnamed: 0',])
combined_draft_df = combined_draft_df.rename(columns = {"Player": "name", "NFL team": "NFL_Team", "Pos.": "Pos", "Conf.": "Conf", "Rnd.": "Round", 
                                      "Pick No.": "Pick_No", "year": "Year"})

In [25]:
#Preview Combined Draft DF
combined_draft_df.head()

,Round,Pick_No,NFL_Team,name,Pos,College,Conf,Year
0,1,1.0,Los Angeles Rams,Jared Goff †,QB,California,Pac-12,2016
1,1,2.0,Philadelphia Eagles,Carson Wentz †,QB,North Dakota State,MVFC,2016
2,1,3.0,San Diego Chargers,Joey Bosa †,DE,Ohio State,Big Ten,2016
3,1,4.0,Dallas Cowboys,Ezekiel Elliott †,RB,Ohio State,Big Ten,2016
4,1,5.0,Jacksonville Jaguars,Jalen Ramsey †,CB,Florida State,ACC,2016


##### Need to get one single dataframe of just the players

In [26]:
#Create player dataframe from combined_combine_df
combine_player_df = pd.DataFrame(combined_combine_df["name"])

In [27]:
#Preview combine_player_df
combine_player_df.head()

,name
0,Mehdi Abdesmad
1,Vernon Adams
2,Jerell Adams
3,Bralon Addison
4,Roberto Aguayo


In [28]:
#Check Number of Players in combine_player_df
len(combine_player_df)

1699

In [31]:
#Create player dataframe from combined_draft_df
draft_player_df = pd.DataFrame(combined_draft_df["name"])

In [32]:
#Preview draft_player_df
draft_player_df.head()

,name
0,Jared Goff †
1,Carson Wentz †
2,Joey Bosa †
3,Ezekiel Elliott †
4,Jalen Ramsey †


In [33]:
#Check Number of Players in draft_player_df
len(draft_player_df)

1282

In [34]:
#Combine both player dataframes
player_frames = [combine_player_df, draft_player_df]
combined_player_df = pd.concat(player_frames)

In [35]:
#Preview combined_player_df
combined_player_df.head()

,name
0,Mehdi Abdesmad
1,Vernon Adams
2,Jerell Adams
3,Bralon Addison
4,Roberto Aguayo


In [36]:
#Check number of players of combined_player_df - This check out.  Is the sum of the combine_player_df and draft_player_df
len(combined_player_df)

2981

In [38]:
#Drop duplicate players in combined_player_df
player_df = combined_player_df.drop_duplicates(subset='name', keep='first', ignore_index=True)

In [39]:
#Preview player_df
player_df.head()

,name
0,Mehdi Abdesmad
1,Vernon Adams
2,Jerell Adams
3,Bralon Addison
4,Roberto Aguayo


In [40]:
#Check number of players in player_df - 2006 players total.  Check out.
len(player_df)

2006

In [41]:
#Preview Player Table
player_df.head()

,name
0,Mehdi Abdesmad
1,Vernon Adams
2,Jerell Adams
3,Bralon Addison
4,Roberto Aguayo


# Load

In [42]:
#Connect to local database
rds_connection_string = f'{username}:{password}@localhost:5432/{database_name}'
engine = create_engine(f'{username}+psycopg2://{rds_connection_string}')

In [ ]:
# Check the tables names to make sure where are we posting
engine.table_names()

In [ ]:
# Load data using pandas. name stands for table name, change it if neccesary.
player_df.to_sql(name='player', con=engine, if_exists='append', index=False)

In [43]:
#Confirm the data has been Load. Check table name. Create a dataframe to get player_id to the other dataframes
player_id_df=pd.read_sql_query('select * from player', con=engine)
player_id_df

,player_id,name
0,1,Mehdi Abdesmad
1,2,Vernon Adams
2,3,Jerell Adams
3,4,Bralon Addison
4,5,Roberto Aguayo
...,...,...
2001,2002,Chris Jackson
2002,2003,Chris Williamson
2003,2004,Sam Sloman
2004,2005,Kyle Hinton


In [44]:
combine_table = player_id_df.merge(combined_combine_df, how='inner', on='name')

In [45]:
combine_table

,player_id,name,Pos,School,Height,Weight,Forty_Yard,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle,Year
0,1,Mehdi Abdesmad,DE,Boston Col.,78,284,5.10,29.5,25,108,7.55,4.62,2016
1,2,Vernon Adams,QB,Oregon,71,200,4.83,29.5,NaN,114,6.82,4.20,2016
2,3,Jerell Adams,TE,South Carolina,77,247,4.64,32.5,NaN,117,7.05,4.31,2016
3,4,Bralon Addison,WR,Oregon,69,197,4.66,34.5,13,116,6.95,4.14,2016
4,5,Roberto Aguayo,K,Florida State,72,207,4.96,NaN,NaN,NaN,NaN,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694,1656,D.J. Wonnum,DL,South Carolina,77,258,4.73,34.5,20,123,7.25,4.44,2020
1695,1657,Dom Wood-Anderson,TE,Tennessee,76,261,4.92,35.0,NaN,119,NaN,NaN,2020
1696,1658,David Woodward,LB,Utah State,74,230,4.79,33.5,16,114,7.34,4.37,2020
1697,1659,Chase Young,DL,Ohio State,77,264,NaN,NaN,NaN,NaN,NaN,NaN,2020


In [46]:
combine_table = combine_table.drop(columns = ['name', 'Pos', 'School', 'Height', 'Weight', 'Year'])

In [47]:
combine_table

,player_id,Forty_Yard,Vertical,Bench,Broad_Jump,Three_Cone,Shuttle
0,1,5.10,29.5,25,108,7.55,4.62
1,2,4.83,29.5,NaN,114,6.82,4.20
2,3,4.64,32.5,NaN,117,7.05,4.31
3,4,4.66,34.5,13,116,6.95,4.14
4,5,4.96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1694,1656,4.73,34.5,20,123,7.25,4.44
1695,1657,4.92,35.0,NaN,119,NaN,NaN
1696,1658,4.79,33.5,16,114,7.34,4.37
1697,1659,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
# merge with combine dataframe to get the serial id.
combine_df_final = pd.merge(combine_table, player_id_df, how='inner', on='name')
combine_df_final

KeyError: 'name'

In [ ]:
# merge with global dataframe to get the serial id to the dataframe.
global_df = pd.merge(global_df, player_id_df, how='inner', on='player')
global_df


In [ ]:
# Create a new dataframe for the info table
player_id = global_df['player_id']
position = global_df['Pos']
height = global_df['height']
weight = global_df['weight']
year = global_df['year']

info_df = pd.DataFrame{
    'player_id':player_id,
    'position': position,
    'height': height,
    'weight': weight,
    'year': year,
}


In [ ]:
# Create a new dataframe for the teams table
#  note: players from the combine doesn't have a team
teams = global_df['NFL team']

In [ ]:
# Clean the combine df final by dropping columns we don't need and renaming id

combine_df_final = combine_df_final.drop(columns = ['school', 'School','player', 'year_y', 'pos', 'Pos', 'year_x'])
combine_df_final = combine_df_final.rename(columns = {'id':'player_id'})

# Convert types to fit in the database
#### we are using to numeric for floats and convert_dtypes method for string
#### Height needs to be string because is in 6 feet 1 inch format. others are float
combine_df_final['Ht'] = combine_df_final['Ht'].convert_dtypes(infer_objects=True, convert_string=True)
combine_df_final['Wt'] = pd.to_numeric(combine_df_final['Wt'], errors='coerce')
combine_df_final['40yd'] = pd.to_numeric(combine_df_final['40yd'], errors='coerce')
combine_df_final['Bench'] = pd.to_numeric(combine_df_final['Bench'], errors='coerce')
combine_df_final['Vertical'] = pd.to_numeric(combine_df_final['Vertical'], errors='coerce')
combine_df_final['Broad Jump'] = pd.to_numeric(combine_df_final['Broad Jump'], errors='coerce')
combine_df_final['3Cone'] = pd.to_numeric(combine_df_final['3Cone'], errors='coerce')
combine_df_final['Shuttle'] = pd.to_numeric(combine_df_final['Shuttle'], errors='coerce')
# Get the info of the dataframe
combine_df_final.info()

In [ ]:
# merge with draft dataframe to get the serial id.
draft_df_final = pd.merge(draft_df, player_id_df, how='inner', on='player')
draft_df_final


In [ ]:
# Clean the draft df final by dropping columns we don't need and renaming id

draft_df_final = draft_df_final.drop(columns = ['school','School','player', 'year_y', 'pos', 'Pos', 'year_x'])
draft_df_final = draft_df_final.rename(columns = {'id':'player_id'})

# # Convert types to fit in the database
### Rnd coulb be either a string or a float because we are not supossed to do any math operation with it. 
draft_df_final['Rnd'] = pd.to_numeric(draft_df_final['Rnd'], errors='coerce')
draft_df_final['Pick_no'] = draft_df_final['Pick_no'].convert_dtypes(infer_objects=True, convert_integer=True)
draft_df_final['Conf'] = draft_df_final['Conf'].convert_dtypes(infer_objects=True, convert_string=True)


draft_df_final.info()

In [ ]:
college_data = pd.read_csv('COLLEGEUNIVMAPINFO.csv')
college_data = college_data.rename(columns = {'NAME':'School'})
college_data

In [ ]:
total_colleges = pd.DataFrame(global_df['School'].unique())
total_colleges

In [ ]:
# Load data using pandas. name stands for table name, change it if neccesary.
combine_df_final.to_sql(name='combine', con=engine, if_exists='append', index=False)

In [ ]:
# Load data using pandas. name stands for table name, change it if neccesary.
info_df.to_sql(name='info', con=engine, if_exists='append', index=False)

In [ ]:
# Load data using pandas. name stands for table name, change it if neccesary.
college_df.to_sql(name='college', con=engine, if_exists='append', index=False)

In [ ]:
# Load data using pandas. name stands for table name, change it if neccesary.
teams_df.to_sql(name='draft', con=engine, if_exists='append', index=False)

In [ ]:
# Check for the database table
combine_id_df=pd.read_sql_query('select * from combine', con=engine)
combine_id_df

In [ ]:
# Check for the database table
draft_id_df=pd.read_sql_query('select * from draft', con=engine)
draft_id_df

In [ ]:
# Database ready to work